# W tym dokumencie:
* Logowanie do systemu i praca zdalna
    * Uwierzytelnienie przy pomocy kluczy ssh
    * Praca zdalna, przekierowanie portów, kopiowanie po ssh
* Elementy pracy w systemie
    * Edycja plików z linii komend
    * Programowanie w BASH, skrypty
    * ?

* Krótko o bazach danych
    * Czym jest baza danych, rodzaje baz
    * Wprowadzenie 
    * SQL wprowadzenie i przykłady

Źródła i literatura:
* https://www.doc.ic.ac.uk/~wjk/UnixIntro/
* https://www.digitalocean.com/community/tutorials/understanding-the-ssh-encryption-and-connection-process
* https://pl.wikipedia.org/wiki/RSA_(kryptografia)
* https://pl.wikipedia.org/wiki/Arytmetyka_modularna

***

# Dlaczego SSH to Secure Shell?
Dzięki SSH łączenie oraz praca na zdalnej maszynie odbywa się w sposób bezpieczny. My używamy SSH za każdym razem gdy łączymy się z serwerem laboratoryjnym info3. SSH wykorzystuje kilka technik szyfrowania w celu zapewnienia bezpieczeństwa. My pobieżnie przedstawimy pewne podstawy.

## Szyfrowanie symetryczne
W czasie trwania sesji komunikacja użytkownika ze zdalna maszyną szyfrowana jest przy pomocy **szyfrowania symetrycznego**. Tzn. obie strony dysponują tym samym kluczem służącym do szyfrowania / deszyfrowania wiadomości. Klucz tworzony jest na potrzeby pojedynczej sesji i jest współtworzony przez klienta i server na początku komunikacji.

## Szyfrowanie asymetryczne
Polega na wykorzystaniu dwóch kluczy, **publicznego** i **prywatnego**. Klucz publiczny jest z założenia jawny i udostępnia się go *światu*, służy do szyfrowania wiadomości. Odczyt tak zaszyfrowanej wiadomości możliwy jest tylko z wykorzystaniem klucza prywatnego. Klucz prywatny pozostaje niejawny i nikomu się go nie udostępnia (nigdy!). Ten typ szyfrowania wykorzystywany jest przez SSH np. w czasie ustalania symetrycznego klucza sesji. Strony wymieniają się tymczasowymi kluczami publicznymi i używają ich do szyfrowania informacji służących do ustalenia klucza sesji (możliwych do odczytu tylko przy użyciu kluczy prywatnych). 

Dla nas istotne jest wykorzystanie szyfrowania asymetrycznego w celu uwierzytelnienie w czasie logowania do zdalnej maszyny. Podejście takie jest wielokrotnie bardziej bezpieczne niż standardowe podejście oparte o hasło i login, ([np. po ataku na ICM nasz admin uniemożliwił logowanie inne niż poprzez klucz prywatny/publiczny odpowiedniej długości](https://zaufanatrzeciastrona.pl/post/ktos-przez-5-miesiecy-podsluchiwal-hasla-uzytkownikow-centrum-obliczeniowego-uw/)) ponieważ w procesie uwierzytelnienia nigdy nie jest przesyłana wrażliwa informacja, którą można by podsłuchać.

Jak to działa? Użytkownik ujawnia swój klucz publiczny (umieszcza go na serwerze). W procesie uwierzytelnienia klucz publiczny służy do skonstruowania *wyzwania*, tj. zaszyfrowanej wiadomości, którą odczytać można jedynie przy pomocy klucza prywatnego. Jeżeli starający się uwierzytelnić dysponuje kluczem prywatnym odpowiada na wyzwanie i może się uwierzytelnić.

### Szyfrowanie RSA (Rivesta-Shamira-Adlemana)
Obecnie najpopularniejszy algorytm szyfrowania asymetrycznego. Jego bezpieczeństwo opiera się na trudności rozkładu dużych liczb na na dzielniki (faktoryzacji).

#### Generowanie kluczy:
1. Wybierz dwie duże liczby pierwsze $p$ i $q$
2. Oblicz $n=p\cdot q$
3. Oblicz $\varphi(n) = (p-1)(q-1)$ (funkcja Eulera)
4. Wybierz liczbę $e$, względnie pierwszą z $\varphi(n)$ i mniejszą od niej.
5. Znajdź $d$ takie, że $d\cdot e \equiv 1 mod(\varphi(n))$. Ten zapis znaczy, że reszta z dzielenia $d\cdot e - 1$ przez $\varphi(n)$ ma być zero.

Klucz **publiczny** to para liczb $(n,e)$, klucz prywatny to $(n, d)$.  

#### Szyfrowanie:
Wiadomość $m$ szyfrujemy tak:  
$c = m^e (mod(n))$

Deszyfrujemy tak:  
$m = c^d (mod(n))$

Przypis:
$c = m^e(mod(n))$ znaczy reszta z dzielenia $m^e$ przez $n$.  
$a \equiv b (mod(n))$ znaczy, że $a-b$ jest podzielne przez $n$ (przystawania modulo).

#### Przykład (Python 3):
1.

In [1]:
p = 17
q = 11
print(' p =', p, '\n', 'q =', q)

 p = 17 
 q = 11


2. $n=p\cdot q$

In [2]:
n = p*q
print(' n =', n)

 n = 187


3. $\varphi(n) = (p-1)(q-1)$ 

In [3]:
phi = (p-1) * (q-1)
print(' phi =', phi)

 phi = 160


4.

In [6]:
import math
e = 23
print(' e =', e)

print (" The gcd of", phi, "and", e, "is : ",end="") 
print (math.gcd(phi,e))

 e = 23
 The gcd of 160 and 23 is : 1


5. $d$ takie, że $d\cdot e \equiv 1 mod(\varphi(n))$  
$d\cdot e-1$ podzielne przez $\varphi(n)$, albo  
$d = {(\varphi(n)\cdot i + 1)}/{e}, ~~ i = 1,2,3, ... $ ma być całkowite:

In [7]:
for i in range(1,10):
    d = (phi * i + 1) / e
    if d == int(d):
        break
d = int(d)
print(" d =", d)

 d = 7


Szyfrujemy teraz tajna wiadomość przy pomocy klucza **publicznego**. Nasza wiadomość to $m=25$, $c = m^e (mod(n))$
:

In [8]:
m = 25
c = m**e % n
print(" m =", m, "zaszyfrowano do c=", c)

 m = 25 zaszyfrowano do c= 49


Deszyfrujemy przy użyciu klucza **prywatnego**, $m = c^d (mod(n))$
(zobaczmy, że $c^d$ to całkiem spora liczba):

In [9]:
m0 = c**d
print(m0)

678223072849


Reszta z dzielenia przez $n$:

In [10]:
m = m0 % n
print("Nasza tajna wiadomośc to: ", m)

Nasza tajna wiadomośc to:  25


## Logowanie przy pomocy klucza RSA
Aby stworzyć parę kluczy rsa wykorzystamy narzędzia dostępne wraz z SSH:  
```ssh-keygen -t rsa -b 4096```  
stworzy dla nas parę kluczy prywatny/publiczny i umieści je w katalogu ukrytym .ssh. W moim przypadku katalog ten zawiera:

In [ ]:
ls ~/.ssh/

Obecnie mam dwa zestawy kluczy. Domyślnu (```id_rsa```) oraz pozwalajacy na uwierzytelnienie na GitHub. Klucz publiczny umieścilem na naszym serwerze laboratoryjnym. osiąga się to poprzez:  
```ssh-copy-id -i <path_to_public_key> <user>@info3.meil.pw.edu.pl```  
W większości przypadków powinno to wystarczyć.

### Przykład:
Utworzymy nowego użytkownika na info3, może to zrobić super-użytkownik ```root```. Służy do tego komenda ```adduser```.

Następnie wygenerujemy parę kluczy i prześlemy klucz publiczny na info3.

System na naszej maszynie wymusza użycie kontekstów, powinny się one dostosować automatycznie. Niestety na razie tak się nie dzieje. Aby móc wykorzystać klucz publiczny musimy zrobić następującą rzecz:  
```chcon -t ssh_home_t .ssh/authorized_keys```   
tzn. dopasować kontekst pliku w, którym zapisany jest nasz klucz publiczny.

## SSH config
Jeżeli pracujemy z wieloma zdalnymi maszynami, dysponujemy więcej niż jednym kluczem (zalecane by nie używać jednego klucza do wszystkiego), wygodnie jest utworzyć plik ```.ssh/config```. Zawierać on będzie informacje ułatwiające łączenie się ze zdalną maszyną. Pojedynczy wpis może wyglądać tak:  
```
Host info3
    HostName info3.meil.pw.edu.pl
    Port 22
    IdentityFile ~/.ssh/tachion
    User sgepner
```
Zdefiniowaliśmy alias ```info3``` dla maszyny laboratoryjnej, określiliśmy port połączenia (port 22 jest domyślny dla SHH, ale admin paranoik (to zaleta nie wada!) może to zmienić), ścieżkę do klucza prywatnego i użytkownika. Teraz zamiast pisać:  
```ssh -i .ssh/tachion sgepner@info3.meil.pw.edu.pl -p 22```   
napiszemy:  
```ssh info3```

## Kopiowanie po SSH
Do przenoszenia plików z i na zdalną maszynę służy komenda ```scp```. Wywołuje się ja na maszynie lokalnej podając parametry połączenia do maszyny zdalnej. Np. by skopiować ```plik.txt``` z mojego katalogu domowego na ```info3``` zrobił bym tak:  
```scp sgepner@info3.meil.pw.edu.pl:/home/sgepner/plik.txt ./```  
podał hasło i oczekiwał rezultatu. Na szczęście dzięki temu, że na zdalnej maszynie umieściłem swój klucz i odpowiednio skonfigurowałem połączenie (```.ssh/congig```) wystarczy:
```scp info3:/home/sgepner/plik.txt ./```, działa nawet auto-uzupełnianie ścieżki i nazw.

Umieszczenie pliku z komputera lokalnego na zdalnym:  
```scp plik info3:/home/sgepner/```

Dodatkowe przełączniki:
* ```-P``` - określenie portu, jeżeli zmieniony
* ```-r``` - rekurencyjnie w odniesieniu do katalogów

## Przekierowanie portów
Pokażemy teraz jak wykorzystując SSH dostać się do usług, które mogą być schowane w sieci wewnętrznej i dostępne jedynie z jej wnętrza lub zamaskować połączenie tak by wyglądało na wykonane z maszyny zdalnej (np. by dostać się do Biblioteki PW).

### Przekierowanie dynamiczne -D
Na początek spróbujemy "podszyć" się pod Politechnikę i dostać się do wykupionych przez PW zasobów książkowych, dostępnych z terenu PW. W tym celu ustawimy *dynamiczne* przekierowanie portów i ustawimy proxy w przeglądarce. Łączymy się z ```info3```:  
```ssh -D 8080 user@serwer```  
muśmy teraz ustawić pośrednik połączenia w przeglądarce.

### Przekierowanie lokalne -L
Chcemy teraz uzyskać dostęp do usługi udostępnionej w wewnętrznej sieci, niedostępne z zewnątrz. W tym celu ustawimy przekierowania portów i doprowadzimy do sytuacji, że usługa zdawać się będzie lokalną dla naszej maszyny.

Schemat: (narysuj coś!)

Załóżmy, że na maszynie ```hyp-19``` (jeden z węzłów GPU naszego klastra HPC) działa server Jupyter Notebook (i liczymy na nim coś co rzeczywiście wymaga dużo mocy i pamięci, a nie tylko marnujemy zasoby), do którego chcielibyśmy się dostać. Standardowo Jupyter działa na *porcie* 8888. Problem jest jednak taki, że węzły dostępne są jedynie z węzła zarządczego *hyperion* a ten z kolei jedynie poprzez węzeł logowania *tachion*, który wychodzi na świat.

1. Ustawimy połączenie do tachion, z przekierowaniem ruchu przez nasz lokalny port 1234 na port 22 hyperiona.  
```ssh user@server -L local_port:target:target_port -Nf```  
(```-Nf``` pozwala przejście połączenia w tło, bez blokowania konsoli.
2. Ustawimy teraz przekierowanie portu, np. 8088 na 8888 hyp-17 przez hyperiona, piszemy:  
```ssh localhost -p 1233 -L 8088:hyp-18:8888 -i .ssh/tachion -Nf```  
Wyjaśnienie: łączymy się na lokalny port 1233 (tam jest hyperion!) i przekierowujemy port 8088 na port maszyny zdalnej hyp-18. W tym przypadku musimy podać klucz, ponieważ .ssh/config nie określa sposobu łączenia z tą maszyną.

### Przekierowanie odwrotne -R
Przydatne gdy chcemy na zewnątrz udostępnić usługę lokalną, np. serwer gry, albo dostęp do własnej maszyny/powłoki z wykrzystaniem pośrednika w postaci dostępnego serwera.
```ssh user@server -R remote_port:local_target:target_port -Nf```
Ustawi to przekierowanie z portu remote_port na target na target port. Pozwoli na dostanie się do maszyny ustanawiającej tunel z serwera:
Rysunek i przykład


# Skrypty

![https://gal.patheticcockroach.com/_data/i/upload/2017/03/18/20170318173200-fe367f59-me.jpg](https://gal.patheticcockroach.com/_data/i/upload/2017/03/18/20170318173200-fe367f59-me.jpg)


## Operacje na plikach
Omówimy teraz (bardzo pobieżnie, jedynie sygnalizując jego istnienie) narzędzie do manipulowania plikami tekstowymi. Narzędzie to przyda nam się w automatyzacji, poprzez skrypty, procesu np. obliczeniowego.

[```sed```](https://pl.wikipedia.org/wiki/Sed_(program)) czyli stream editor, edytor pozwalający na manipulowanie tekstem, który okazuje się też być językiem programowania! ```sed``` pozwala na użycie wyrażeń regularnych. Mi ```sed``` przydaje się jako narzędzie do automatyzacji operacji na plikach, takiej jak podmiany wartości w plikach konfiguracyjnych. 

```sed -i 's/fraza/fraza2/' plik``` - podmień pierwsze wystąpienie ```fraza``` na ```fraza2``` i nadpisz ```plik```.

```sed 's/fraza/fraza2/' plik > plik2``` - jak wyżej, ale zapis do ```plik2```.

```sed -i "/<P> Re            = 750/ s/750/$re/" sett.xml```

## Programowanie w BASH
Postaramy się teraz zaprezentować kilka elementów powłoki Bash, które pozwalają na automatyzacje zadań. Docelowo będziemy chcieli stworzyć skrypt przygotowujący drzewo katalogowe obliczeń do studium parametrycznego modelu, powiedzmy, że do optymalizacji.

Zaczniemy od tego, że w powłoce użytkownik może definiować zmienne, oraz, że kilka takich zmiennych już istnieje, np PATH, HOME ale i inne. Zmienne używa się ze znakiem \$:

In [ ]:
echo $PATH

In [ ]:
echo $HOME

Możemy też zdefiniować własne:

In [ ]:
nowa_zmienna=dowolny_kawalek_tekstu
echo nowa_zmienne zawiera: $nowa_zmienna

Napiszemy teraz instrukcję zawierają pojedyncze polecenie, ```echo``` (służy do wyświetlania):

In [ ]:
echo Hello!

a jeżeli zapiszemy to w pliku i nadamy mu prawo wykonania uzyskamy prosty skrypt postaci:  

```
#!/bin/bash
# komentarz

echo Hello!!
```

```#``` oznacza komentarz, natomiast ```#!``` informuje system, że do interpretacji wykorzystamy bash.

In [ ]:
ls -l

Nadajemy prawa wykonania:

In [ ]:
chmod u+x skrypt

I możemy go wywołać:

In [ ]:
./skrypt

### Argumenty
Skrypt można wywołać z argumentami, oraz ustalać zmienne. Obie kategorie dostępne są poprzez operator \$  

* \$\$ - PID procesu
* \\$0 - nazwa pliku, w którym umieszczony jest skrypt
* \\$1, \\$2, ... - pierwszy i kolejne argumenty  
* \$\# - liczba argumentów

In [ ]:
./skrypt 1 2 1000

Możemy, np. napisać skrypt, który będzie sam siebie modyfikował ...

### Prawie jak programowanie:
#### Instrukcje warunkowe

```
if [ test ]
then
    what-if
else
    what-else
fi
```

Uwaga: $[$ i $]$ muszą być wprowadzone ze spacją ponieważ są to polecenia powłoki znajdujące się w ```/bin```. Testować możemy wiele rzeczy.

* -s file - czy plik istnieje i nie jest pusty
* -f file - czy plik jest zwykły
* -d file - czy plik jest katalogiem
* -r, -w, -x - czy mozliwy odczyt / zapis/ wykonanie
* Porównanie:
    * -eq - równość
    * -ne - nierówność
    * -lt, gt, -le, -ge, - ...
* inne do znalezienia w razie potrzeby

Uwaga: znaki $>$ i $<$ są niedostępne (dlaczego?) dla wykonania testów logicznych.

By wykorzystać instrukcje warunkowe, czy pętle (zaraz) nie musimy koniecznie pisać skryptu. Możemy wywołać je wprost z linii komend. np: (konieczne mogą być średniki)

In [ ]:
if [ ! -f burza ]; then echo Ala; else echo ma kota; fi

#### Pętle for i while
```
for i in list
do
    zadania
done
```

```
while [ test ]
do
    zadania
done
```

In [ ]:
$(seq $1 $3 $2)

# Bazy danych
Zorganizowany zbiór danych, obecnie przeważnie w formie elektronicznej, dostępny przy użyciu odpowiedniego programu (DBMS). Bazy danych klasyfikuje się, ze względu na model przechowywania danych. My wykorzystamy najczęściej używany typ bazy tj. *relacyjną*. W tym podejściu dane przechowywane są jako *rzędy* i *kolumny* w *tabelach* połączonych ze sobą *relacjami*. Operacje i zapytania najczęściej wykonuje się z użyciem SQL (Structured Query Language). Istnieją też inne modele baz (całkiem dużo) my wspomnimy tylko bazy nierelacyjne (NoSQL), które stały się popularne w zastosowaniach do aplikacji sieciowych i przetwarzania danych.